# Clean up downloaded data and merge with other data sources

In [1]:
import pandas as pd
from json import load
import urllib.request, json 
from pandas.io.json import json_normalize
import seaborn as sns
import pylab as plt
import multiprocessing as mp
import numpy as np
from sklearn.manifold import TSNE
from pathlib import Path
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from urllib.error import HTTPError
from time import sleep

In [2]:
def clean_dollarsigns(df):
    replacements = {'_created.$date':'_created', '_id.$oid':'_id', '_updated.$date':'_updated'}
    df = df.rename(columns=replacements)
    return df

## Load csvs with all MRIQC data
Either downloaded through the api, from Chris Gorgolewski's Kaggle page: https://www.kaggle.com/chrisfilo/mriqc or from the OSF directory OHBM2019-TSNE-Analysis in the MRIQC project directory here: https://osf.io/haf97/. The ALL.txt file use here contains hashes and dataset identies for files from semi-public datasets we had access to locally. It is also available in the project directory on OSF.

In [3]:
# load csvs
datestr="2019_05_22"

df_t1w = pd.read_csv(f'all_t1w_{datestr}.csv', index_col=0, low_memory=False)
df_t2w = pd.read_csv(f'all_t2w_{datestr}.csv', index_col=0, low_memory=False)
df_bold = pd.read_csv(f'all_bold_{datestr}.csv', index_col=0, low_memory=False)

df_t1w = clean_dollarsigns(df_t1w)
df_t2w = clean_dollarsigns(df_t2w)
df_bold = clean_dollarsigns(df_bold)

/data/MLcore/miniconda3/envs/mriqc_tsne/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
# load extra data from local runs of mriqc
json_dir = Path('jsons/')
HCP_dir = json_dir / 'HCP'
SALD_dir = json_dir / 'SALD'
NNDSP_dir = json_dir / 'NNDSP'

for ds, ds_dir in [('hcp', HCP_dir), ('sald', SALD_dir), ('nndsp', NNDSP_dir)]:
    
    dat = []
    for js in ds_dir.glob('*_T1w.json'):
        dat.append(json_normalize(json.loads(js.read_text())))
    dat = pd.concat(dat, ignore_index=True, sort=False)
    dat['dataset_lr'] = ds
    df_t1w = pd.concat([df_t1w, dat], sort=False)

for ds, ds_dir in [('nndsp', NNDSP_dir)]:
    
    dat = []
    for js in ds_dir.glob('*_bold.json'):
        dat.append(json_normalize(json.loads(js.read_text())))
    dat = pd.concat(dat, ignore_index=True, sort=False)
    dat['dataset_lr'] = ds
    df_bold = pd.concat([df_bold, dat], sort=False)

Count up the number of null fields and drop duplicates while keeping the rows with the fewest nans

In [5]:
df_t1w['null_count'] = df_t1w.isnull().sum(1)
df_t2w['null_count'] = df_t2w.isnull().sum(1)
df_bold['null_count'] = df_bold.isnull().sum(1)

# Sort so that rows with fewest nulls and most recent creation are towards the top
df_t1w_unique = df_t1w.sort_values(['null_count','_created'], ascending=[True, False]).drop_duplicates(subset=['provenance.md5sum'])
df_t2w_unique = df_t2w.sort_values(['null_count','_created'], ascending=[True, False]).drop_duplicates(subset=['provenance.md5sum'])
df_bold_unique = df_bold.sort_values(['null_count','_created'], ascending=[True, False]).drop_duplicates(subset=['provenance.md5sum'])

In [6]:
print(df_t1w.shape, df_t1w_unique.shape)
print(df_bold.shape, df_bold_unique.shape)

(1134913, 244) (117884, 244)
(303089, 229) (165500, 229)


Load up datalad metadata extracted on 05-23-2019, see the get_datalad_data notebook to prepare this data yourself.

In [65]:
dl_df = pd.read_csv('datalad_metadata.csv', low_memory=False)
dl_df['hash'] = dl_df.loc[dl_df.metadata__annex__key.notnull(),'metadata__annex__key'].str.split('--').str[-1].str.split('.').str[0]
dl_df['hashing_algo'] = dl_df.loc[dl_df.metadata__annex__key.notnull(),'metadata__annex__key'].str.split('--').str[0].str.split('-').str[0]
dl_df=dl_df.drop_duplicates(subset=['hashing_algo', 'hash'])

df_t1w_dl_merge = df_t1w_unique.loc[~df_t1w_unique['provenance.settings.testing'], :].merge(dl_df.loc[dl_df.hashing_algo == 'MD5E', :], indicator=True, how='left', left_on='provenance.md5sum', right_on='hash')
df_t2w_dl_merge = df_t2w_unique.loc[~df_t2w_unique['provenance.settings.testing'], :].merge(dl_df.loc[dl_df.hashing_algo == 'MD5E', :], indicator=True, how='left', left_on='provenance.md5sum', right_on='hash')
df_bold_dl_merge = df_bold_unique.merge(dl_df.loc[dl_df.hashing_algo == 'MD5E', :], indicator=True, how='left', left_on='provenance.md5sum', right_on='hash')

In [66]:
df_t1w_dl_merge.groupby('_merge')[['_etag']].count()


,_etag
_merge,
left_only,111701
right_only,0
both,4428


In [67]:
df_t2w_dl_merge.groupby('_merge')[['_etag']].count()

,_etag
_merge,
left_only,14688
right_only,0
both,368


In [68]:
df_bold_dl_merge.groupby('_merge')[['_etag']].count()

,_etag
_merge,
left_only,151732
right_only,0
both,12610


Load hashes calculated on local copies of semi-public datasets

In [96]:
ukbb = pd.concat([
    pd.read_csv('ukbb_t1_md5.txt',
                names=['hash', 'path'],
                delim_whitespace=True),
    pd.read_csv('ukbb_bold_md5.txt',
                names=['hash', 'path'],
                delim_whitespace=True)],
    ignore_index=True)
ukbb['dataset'] = 'UKBB'

abcd = pd.concat([
    pd.read_csv('abcd_t1_md5.txt',
                names=['hash', 'path'],
                delim_whitespace=True),
    pd.read_csv('abcd_bold_md5.txt',
                names=['hash', 'path'],
                delim_whitespace=True)],
    ignore_index=True)
abcd['dataset'] = 'ABCD'

nild = pd.read_csv('dnude_t1_md5.txt',
                names=['hash', 'path'],
                delim_whitespace=True)
nild['dataset'] = 'NILD'

dsst_datasets = pd.read_csv('ALL.txt',
                names=['dataset', 'hash', 'path'],
                delim_whitespace=True)

dsst_datasets = pd.concat([dsst_datasets, ukbb,abcd, nild], ignore_index=True, sort=False)
dsst_datasets.drop_duplicates('hash', inplace=True)
# make sure that there aren't any hashes assigned to more than one data set
assert dsst_datasets.groupby('hash').dataset.nunique().head().max() == 1

In [97]:
print(dsst_datasets.dataset.unique())
df_t1w_merge = df_t1w_dl_merge.merge(dsst_datasets, how='left', left_on='provenance.md5sum', right_on='hash', suffixes=('', '_HPC'), indicator='_dsst')
df_t2w_merge = df_t2w_dl_merge.merge(dsst_datasets, how='left', left_on='provenance.md5sum', right_on='hash', suffixes=('', '_HPC'), indicator='_dsst')
df_bold_merge = df_bold_dl_merge.merge(dsst_datasets, how='left', left_on='provenance.md5sum', right_on='hash', suffixes=('', '_HPC'), indicator='_dsst')

['Abide2' 'Abide' 'adhd200' 'CMI' 'CoRR' 'fcon1000' 'HCP' 'NNDSP'
 'OpenNeuro' 'SALD' 'UKBB' 'ABCD' 'NILD']


 Define functions for cleaning up some of the bids meta fields

In [98]:
def clean_factor(df, new_names, column):
    mlist = []
    bad = []
    for old_name in df[column]:
        try:
            mlist.append(new_names[old_name])
        except KeyError:
            if pd.notnull(old_name):
                bad.append(old_name)
            mlist.append(np.nan)
    return bad, mlist

model_dict = {'Signa HDe': 'Signa HDe',
              'Signa_HDxt': 'Signa HDxt',
              'Signa HDxt': 'Signa HDxt',
              'Signa HDx' : 'Signa HDx',
              'SIGNA_HDx': 'Signa HDx',
              'SIGNA_PET_MR': 'Signa PET-MR',
              'SIGNA_EXCITE': 'Signa Excite',
              'Signa Twin Speed Excite HD scanne': 'Signa Excite',
              'SIGNA_Premier': 'Signa Premier',
              'Signa Premier': 'Signa Premier',
              'Signa': 'Signa',
              'SIGNA_Creator': 'Signa Creator',
              'GENESIS_SIGNA': 'Signa Genesis',
              'Symphony': 'Symphony',
              'SymphonyTim': 'Symphony',
              'Tim TRIO': 'Tim Trio',
              'TrioTim': 'Tim Trio',
              'Magnetom Trio' : 'Tim Trio',
              'TIM TRIO': 'Tim Trio',
              'MAGNETOM Trio': 'Tim Trio',
              'TRIOTIM': 'Tim Trio',
              'Trio': 'Tim Trio',
              'Trio TIM': 'Tim Trio',
              'Tim Trio': 'Tim Trio',
              'TRIO': 'Tim Trio',
              'MAGNETOM Trio A Tim': 'Tim Trio',
              'Magentom': 'Tim Trio',
              'TimTrio': 'Tim Trio',
              'TriTim': 'Tim Trio',
              'MAGNETOM and Jerry': 'MAGNETOM and Jerry',
              'Trio Magnetom': 'Tim Trio',
              'Prisma_fit': 'Prisma',
              'Prisma': 'Prisma',
              'Magnetom Skyra Fit': 'Skyra',
              'MAGNETOM Skyra': 'Skyra',
              'Skyra': 'Skyra',
              'Intera': 'Intera',
              'Allegra': 'Allegra',
              'Verio': 'Verio',
              'Avanto': 'Avanto',
              'Sonata': 'Sonata',
              'Espree': 'Espree',
              'SonataVision': 'Sonata Vision',
              'Spectra':'Spectra',
              'Ingenia' : 'Ingenia',
              'Ingenia_CX': 'Ingenia CX',
              'DISCOVERY MR750': 'Discovery MR750',
              'DISCOVERY_MR750': 'Discovery MR750',
              'DISCOVERY_MR750w': 'Discovery MR750',
              'Discovery MR750': 'Discovery MR750',
              'MR750': 'Discovery MR750',
              'Achieva_dStream': 'Achieva dStream',
              'Achieva dStream': 'Achieva dStream',
              'Achieva Ds': 'Achieva dStream',
              'Achieva': 'Achieva',
              'Achieva TX': 'Achieva TX',
              'Intera_Achieva': 'Achieva',
              'Intera Achieva': 'Achieva',
              'Philips Achieva': 'Achieva',
              'GEMINI': 'Gemini',
              'Ingenuity': 'Ingenuity',
              'Gyroscan_Intera': 'Gyroscan Intera',
              'Biograph_mMR': 'Biograph mMR',
              'NUMARIS_4': 'Numaris 4',
              'Investigational_Device_7T': 'Investigational 7T',
              'N/A': np.nan,
              '': np.nan,
              'DicomCleaner': np.nan,
              'Orchestra_SDK': np.nan}

mfg_dict = {'Siemens': 'Siemens',
            'SIEMENS': 'Siemens',
            'Simiens': 'Siemens',
            'Siemans': 'Siemens',
            'Simens': 'Siemens',
            'GE': 'GE',
            'G.E.': 'GE',
            'GE MEDICAL SYSTEMS': 'GE',
            'GE_MEDICAL_SYSTEMS': 'GE',
            'General Electric': 'GE',
            'General Electrics': 'GE',
            'GE 3 Tesla MR750': 'GE',
            'Philips':'Philips',
            'Philips Ingenia 3.0T': 'Philips',
            'Philips Achieva Intera 3 T Scanner': 'Philips', 
            'Philips Medical Systems': 'Philips',
            'Bruker BioSpin MRI GmbH': 'Bruker'
           }

def clean_table(res_df):
    # find all the rows with a mfg of 'GE 3 Tesla MR750' and make sure they've got a model value
    res_df.loc[res_df['bids_meta.Manufacturer'] == 'GE 3 Tesla MR750', 'bids_meta.ManufacturersModelName'] = 'Discovery MR750'
    res_df.loc[res_df['bids_meta.Manufacturer'] == 'GE 3 Tesla MR750', 'bids_meta.MagneticFieldStrength'] = 3.0
    res_df.loc[res_df['bids_meta.Manufacturer'] == 'Philips Ingenia 3.0T', 'bids_meta.ManufacturersModelName'] = 'Ingenia'
    res_df.loc[res_df['bids_meta.Manufacturer'] == 'Philips Ingenia 3.0T', 'bids_meta.MagneticFieldStrength'] = 3.0
    res_df.loc[res_df['bids_meta.Manufacturer'] == 'Philips Achieva Intera 3 T Scanner', 'bids_meta.ManufacturersModelName'] = 'Achieva'
    res_df.loc[res_df['bids_meta.Manufacturer'] == 'Philips Achieva Intera 3 T Scanner', 'bids_meta.MagneticFieldStrength'] = 3.0

    bad, mlist = clean_factor(res_df, model_dict, 'bids_meta.ManufacturersModelName')
    assert len(pd.unique(bad)) == 0
    res_df['bids_meta.ManufacturersModelName'] = mlist

    bad, mlist = clean_factor(res_df, mfg_dict , 'bids_meta.Manufacturer')
    assert len(pd.unique(bad)) == 0
    res_df['bids_meta.Manufacturer'] = mlist
    
    res_df['dataset_dl'] = res_df.path.str.split('/').str[6]
    res_df['subdataset_dl'] = res_df.path.str.split('/').str[7]
    res_df.loc[(res_df.dataset_dl == "indi"), 'subdataset_dl'] = res_df.path.str.split('/').str[7:9].str.join('__')
    res_df['dataset'] = res_df.dataset.str.lower()
    res_df.dataset = res_df.dataset.fillna(res_df.dataset_dl)
    try:
        res_df.dataset = res_df.dataset.fillna(res_df.dataset_lr)
    except AttributeError:
        pass
    res_df.loc[res_df.dataset == 'openneuro', 'subdataset'] = res_df.path_HPC.str.split('/').str[4]
    res_df.subdataset = res_df.subdataset.fillna(res_df.subdataset_dl)
    return res_df

# Experimenting with pulling information out of additional fields

# xs = ['bids_meta.PulseSequenceType', 'bids_meta.ScanningSequence',
#       'bids_meta.SequenceVariant', 'bids_meta.ScanOptions',
#       'bids_meta.PulseSequenceDetails', 'bids_meta.ScanOptions']
# for x in xs:
#     print(x, res_df[x].str.upper().unique(), res_df[x].notnull().sum())

# #based on http://mriquestions.com/commercial-acronyms.html
# sequence_dict = {'RF-Spoiled GRE': ((res_df['bids_meta.ProtocolName'].str.upper().str.contains('FLASH'))
#                                | (res_df['bids_meta.ProtocolName'].str.upper().str.contains('SPGR'))
#                                | (res_df['bids_meta.ProtocolName'].str.upper().str.contains('FFE'))),
#               'Ultrafast GRE': ((res_df['bids_meta.ProtocolName'].str.upper().str.contains('RAGE'))
#                                 | (res_df['bids_meta.ProtocolName'].str.upper().str.contains('MPR'))
#                                 | (res_df['bids_meta.ProtocolName'].str.upper().str.contains('BRAVO'))
#                                 | (res_df['bids_meta.ProtocolName'].str.upper().str.contains('TFE')))}
# ssq_dict = {'RF-Spoiled GRE': ((res_df['bids_meta.ScanningSequence'] == 'IRSPGR')
#                                   | (res_df['bids_meta.ScanningSequence'] == '3D SPGR')
#                                   | (res_df['bids_meta.ScanningSequence'] == 'IR-FSPGR')),
#              'Ultrafast GRE': ((res_df['bids_meta.ScanningSequence'] == 'MPRAGE')
#                                   | (res_df['bids_meta.ScanningSequence'] == 'T1 turbo field echo')
#                                   | (res_df['bids_meta.ScanningSequence'] == '3D TFE')) }

# res_df['clean_sequence_name'] = np.nan
# for name,ind in sequence_dict.items():
#     res_df.loc[ind,'clean_sequence_name'] = name
# for name,ind in ssq_dict.items():
#     res_df.loc[ind,'clean_sequence_name'] = name


# #cp_coil is single channel reference: https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.20925
# coil_dict = {8: (res_df['bids_meta.ReceiveCoilName'].str.contains('8')
#                 & res_df['bids_meta.ReceiveCoilName'].notnull()),
#              12: ((res_df['bids_meta.ReceiveCoilName'].str.contains('12')
#                   | res_df['bids_meta.ReceiveCoilName'].str.contains('HeadMatrix'))
#                  & res_df['bids_meta.ReceiveCoilName'].notnull()),
#              16: (res_df['bids_meta.ReceiveCoilName'].str.contains('16')
#                  & res_df['bids_meta.ReceiveCoilName'].notnull()),
#              20: (res_df['bids_meta.ReceiveCoilName'].str.contains('20')
#                  & res_df['bids_meta.ReceiveCoilName'].notnull()),
#              32: (res_df['bids_meta.ReceiveCoilName'].str.contains('32')
#                  & res_df['bids_meta.ReceiveCoilName'].notnull()),
#              64: (res_df['bids_meta.ReceiveCoilName'].str.contains('64')
#                  & res_df['bids_meta.ReceiveCoilName'].notnull()),
#              1: (res_df['bids_meta.ReceiveCoilName'].str.contains('CP')
#                 & res_df['bids_meta.ReceiveCoilName'].notnull())}
# for name,ind in coil_dict.items():
#     res_df.loc[ind,'clean_receive_channels'] = name

In [99]:
df_t1w_merge = clean_table(df_t1w_merge)
df_t2w_merge = clean_table(df_t2w_merge)
df_bold_merge = clean_table(df_bold_merge)

In [100]:
df_t1w_merge.to_csv('t1w_datalad_and_mriqc.csv', index=None)
df_t2w_merge.to_csv('t2w_datalad_and_mriqc.csv', index=None)
df_bold_merge.to_csv('bold_datalad_and_mriqc.csv', index=None)

In [101]:
df_t1w_merge.dataset.notnull().sum(), df_bold_merge.dataset.notnull().sum()

(21759, 66400)

In [102]:
df_t1w_merge.groupby('dataset')['provenance.md5sum'].count()

dataset
abcd         8586
abide        1085
abide2       1259
adhd200       950
cmi          1795
corr          202
fcon1000       38
hcp          1054
nild         1499
nndsp         494
openfmri      791
openneuro    3799
sald          207
Name: provenance.md5sum, dtype: int64

In [20]:
# old result df_t1w_merge.groupby('dataset')['provenance.md5sum'].count()

dataset
abide        1123
abide2       1259
cmi           982
corr           41
fcon1000       51
hcp          1177
nndsp         495
openfmri      766
openneuro    3076
sald          207
Name: provenance.md5sum, dtype: int64

In [103]:
df_bold_merge.groupby('dataset')['provenance.md5sum'].count()

dataset
abcd         37696
abide         1139
abide2        1312
adhd200         83
cmi           6403
corr          5030
fcon1000       497
nndsp         1158
openfmri      1615
openneuro    11467
Name: provenance.md5sum, dtype: int64

In [21]:
# Old result df_bold_merge.groupby('dataset')['provenance.md5sum'].count()

dataset
abide        1162
abide2       1152
cmi          2014
fcon1000      497
nndsp        1168
openfmri     1330
openneuro    9675
Name: provenance.md5sum, dtype: int64